In [2]:
! code .

## document
http://www.open3d.org/

## example

### ポイントクラウド
このチュートリアルでは、ポイントクラウドの基本的な使い方を示します。

In [9]:
data_ply = "TestData/fragment.ply"
data_ply = "sync.ply"

In [10]:
import numpy as np
from open3d import *

if __name__ == "__main__":

    print("Load a ply point cloud, print it, and render it")
    pcd = read_point_cloud(data_ply)
    print(pcd)
    print(np.asarray(pcd.points))
    draw_geometries([pcd])

    print("Downsample the point cloud with a voxel of 0.05")
    downpcd = voxel_down_sample(pcd, voxel_size = 0.05)
    draw_geometries([downpcd])

    print("Recompute the normal of the downsampled point cloud")
    estimate_normals(downpcd, search_param = KDTreeSearchParamHybrid(
            radius = 0.1, max_nn = 30))
    draw_geometries([downpcd])

    print("Print a normal vector of the 0th point")
    print(downpcd.normals[0])
    print("Print the normal vectors of the first 10 points")
    print(np.asarray(downpcd.normals)[:10,:])
    print("")

    print("Load a polygon volume and use it to crop the original point cloud")
    vol = read_selection_polygon_volume("TestData/Crop/cropped.json")
    chair = vol.crop_point_cloud(pcd)
    draw_geometries([chair])
    print("")

    print("Paint chair")
    chair.paint_uniform_color([1, 0.706, 0])
    draw_geometries([chair])
    print("")

Load a ply point cloud, print it, and render it
PointCloud with 160801 points.
[[-3.         -3.          0.17846472]
 [-2.985      -3.          0.17440115]
 [-2.97       -3.          0.17063709]
 ...
 [ 2.97        3.          0.17063709]
 [ 2.985       3.          0.17440115]
 [ 3.          3.          0.17846472]]
Downsample the point cloud with a voxel of 0.05
Recompute the normal of the downsampled point cloud
Print a normal vector of the 0th point
[0.18925442 0.21238278 0.95868468]
Print the normal vectors of the first 10 points
[[ 0.18925442  0.21238278  0.95868468]
 [ 0.14832633 -0.01053055  0.9888824 ]
 [ 0.08089142 -0.06675276 -0.99448512]
 [ 0.68583492 -0.52396149 -0.50506912]
 [ 0.26488907  0.34997715  0.89852645]
 [ 0.2611241   0.21195618  0.94174773]
 [ 0.13001961  0.03876958 -0.99075316]
 [ 0.28799628  0.57241931  0.76772018]
 [ 0.08585969  0.23150057  0.96903849]
 [ 0.20777711  0.20212181  0.95706606]]

Load a polygon volume and use it to crop the original point cloud




チュートリアルの最初の部分では、ポイントクラウドを読み取り、視覚化します。

In [13]:
    print("Load a ply point cloud, print it, and render it")
    pcd = read_point_cloud(data_ply)
    print(pcd)
    print(np.asarray(pcd.points))
    draw_geometries([pcd])

Load a ply point cloud, print it, and render it
PointCloud with 160801 points.
[[-3.         -3.          0.17846472]
 [-2.985      -3.          0.17440115]
 [-2.97       -3.          0.17063709]
 ...
 [ 2.97        3.          0.17063709]
 [ 2.985       3.          0.17440115]
 [ 3.          3.          0.17846472]]


read_point_cloudファイルから点群を読み込みます。拡張子名に基づいてファイルをデコードしようとします。サポートの拡張名は次のとおりです。pcd、ply、xyz、xyzrgb、xyzn、pts。

draw_geometries点群を可視化します。マウス/トラックパッドを使用して、異なる視点からジオメトリを表示します。

それは密な表面のように見えますが、実際にはサーフェスとしてレンダリングされる点群です。GUIは、さまざまなキーボード機能をサポートしています。そのうちの1つは-、ポイント（サーフェル）のサイズを小さくするキーです。複数回押すと、視覚化は次のようになります。

### ボクセルダウンサンプリング

ボクセルのダウンサンプリングは、通常のボクセルグリッドを使用して、入力点群から一様にダウンサンプリングされた点群を作成します。これは、多くのポイントクラウド処理タスクの前処理ステップとしてよく使用されます。アルゴリズムは2つのステップで動作します。

1. ポイントはボクセルに入れられます。
1. 占有された各ボクセルは、内部のすべての点を平均化することによって正確な1点を生成する。

これは、ダウンサンプリングされたポイントクラウドです。

In [14]:
    print("Downsample the point cloud with a voxel of 0.05")
    downpcd = voxel_down_sample(pcd, voxel_size = 0.05)
    draw_geometries([downpcd])

Downsample the point cloud with a voxel of 0.05


### 頂点法線推定
ポイントクラウドのもう1つの基本的な操作は、点法線推定です。

In [15]:
    print("Recompute the normal of the downsampled point cloud")
    estimate_normals(downpcd, search_param = KDTreeSearchParamHybrid(
            radius = 0.1, max_nn = 30))
    draw_geometries([downpcd])

Recompute the normal of the downsampled point cloud


estimate_normals各点の法線を計算します。この関数は、共分散解析を使用して隣接点を見つけ、隣接点の主軸を計算します。

この関数は、KDTreeSearchParamHybridclassのインスタンスを引数として取ります。2つの重要な引数で、検索半径と最大最近傍を指定します。それは10cmの検索半径を持ち、計算時間を節約するために最大30人の近傍しか考慮しません。radius = 0.1max_nn = 30

注意

共分散分析アルゴリズムは、通常の候補とは反対の2つの方向を生成する。ジオメトリのグローバルな構造を知らなければ、どちらも正しいことがあります。これは、正常な向きの問題として知られています。Open3Dは、法線が存在する場合、元の法線と揃うように方向付けを試みます。それ以外の場合、Open3Dはランダムな推測を行います。など、さらにオリエンテーション機能orient_normals_to_align_with_directionとはorient_normals_towards_camera_location向きが懸念される場合に呼び出される必要があります。

使用するdraw_geometries点群とプレスを視覚化するためにn、通常のポイントを参照してください。キー-とキー+を使用して法線の長さを制御することができます。

### 推定頂点法線にアクセス
推定された法線ベクトルはのnormals変数によって取り出すことができますdownpcd。

In [16]:
    print("Print a normal vector of the 0th point")
    print(downpcd.normals[0])

Print a normal vector of the 0th point
[0.18925442 0.21238278 0.95868468]


他の変数をチェックするには、使用してくださいhelp(downpcd)。法線ベクトルは、を使用してnumpy配列として変換できますnp.asarray。



In [17]:
    print("Print the normal vectors of the first 10 points")
    print(np.asarray(downpcd.normals)[:10,:])

Print the normal vectors of the first 10 points
[[ 0.18925442  0.21238278  0.95868468]
 [ 0.14832633 -0.01053055  0.9888824 ]
 [ 0.08089142 -0.06675276 -0.99448512]
 [ 0.68583492 -0.52396149 -0.50506912]
 [ 0.26488907  0.34997715  0.89852645]
 [ 0.2611241   0.21195618  0.94174773]
 [ 0.13001961  0.03876958 -0.99075316]
 [ 0.28799628  0.57241931  0.76772018]
 [ 0.08585969  0.23150057  0.96903849]
 [ 0.20777711  0.20212181  0.95706606]]


numpy配列に関するその他の例は、NumPyの使用を参照してください。

### 作物点群

In [18]:
    print("Load a polygon volume and use it to crop the original point cloud")
    vol = read_selection_polygon_volume("TestData/Crop/cropped.json")
    chair = vol.crop_point_cloud(pcd)
    draw_geometries([chair])
    print("")

Load a polygon volume and use it to crop the original point cloud



read_selection_polygon_volumeポリゴン選択領域を指定するjsonファイルを読み込みます。 vol.crop_point_cloud(pcd)ポイントを除外します。椅子だけが残っています。

### ペイントポイントクラウド

In [19]:
    print("Paint chair")
    chair.paint_uniform_color([1, 0.706, 0])
    draw_geometries([chair])
    print("")

Paint chair



paint_uniform_colorすべての点を均一な色に塗りつぶします。色はRGB空間の[0、1]の範囲です。

## メッシュ
Open3Dには、三角形メッシュのデータ構造があります。

In [ ]:
data_ply "TestData/knot.ply"

In [20]:
# examples/Python/Tutorial/Basic/mesh.py

import copy
import numpy as np
from open3d import *

if __name__ == "__main__":

    print("Testing mesh in open3d ...")
    mesh = read_triangle_mesh(data_ply)
    print(mesh)
    print(np.asarray(mesh.vertices))
    print(np.asarray(mesh.triangles))
    print("")

    print("Try to render a mesh with normals (exist: " +
            str(mesh.has_vertex_normals()) +
            ") and colors (exist: " + str(mesh.has_vertex_colors()) + ")")
    draw_geometries([mesh])
    print("A mesh with no normals and no colors does not seem good.")

    print("Computing normal and rendering it.")
    mesh.compute_vertex_normals()
    print(np.asarray(mesh.triangle_normals))
    draw_geometries([mesh])

    print("We make a partial mesh of only the first half triangles.")
    mesh1 = copy.deepcopy(mesh)
    mesh1.triangles = Vector3iVector(
            np.asarray(mesh1.triangles)[:len(mesh1.triangles)//2, :])
    mesh1.triangle_normals = Vector3dVector(
            np.asarray(mesh1.triangle_normals)
            [:len(mesh1.triangle_normals)//2, :])
    print(mesh1.triangles)
    draw_geometries([mesh1])

    print("Painting the mesh")
    mesh1.paint_uniform_color([1, 0.706, 0])
    draw_geometries([mesh1])


Testing mesh in open3d ...
TriangleMesh with 160801 points and 0 triangles.
[[-3.         -3.          0.17846472]
 [-2.985      -3.          0.17440115]
 [-2.97       -3.          0.17063709]
 ...
 [ 2.97        3.          0.17063709]
 [ 2.985       3.          0.17440115]
 [ 3.          3.          0.17846472]]
[]

Try to render a mesh with normals (exist: False) and colors (exist: False)
A mesh with no normals and no colors does not seem good.
Computing normal and rendering it.
[]
We make a partial mesh of only the first half triangles.
std::vector<Eigen::Vector3i> with 0 elements.
Use numpy.asarray() to access data.
Painting the mesh


## NumPyを使って作業する
Open3Dのデータ構造は、NumPyバッファとネイティブに互換性があります。次のチュートリアルでは、NumPyを使用した同期関数の変形を生成し、Open3Dを使用してその関数を視覚化します。



In [1]:
# examples/Python/Tutorial/Basic/working_with_numpy.py

import copy
import numpy as np
from open3d import *


# generate some neat n times 3 matrix using a variant of sync function
x = np.linspace(-3, 3, 401)
mesh_x, mesh_y = np.meshgrid(x,x)
z = np.sinc((np.power(mesh_x,2)+np.power(mesh_y,2)))
z_norm = (z-z.min())/(z.max()-z.min())
xyz = np.zeros((np.size(mesh_x),3))
xyz[:,0] = np.reshape(mesh_x,-1)
xyz[:,1] = np.reshape(mesh_y,-1)
xyz[:,2] = np.reshape(z_norm,-1)
print('xyz')
print(xyz)


xyz
[[-3.         -3.          0.17846472]
 [-2.985      -3.          0.17440115]
 [-2.97       -3.          0.17063709]
 ...
 [ 2.97        3.          0.17063709]
 [ 2.985       3.          0.17440115]
 [ 3.          3.          0.17846472]]


In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame(xyz)

In [5]:

# Pass xyz to Open3D.PointCloud and visualize
pcd = PointCloud()
pcd.points = Vector3dVector(xyz)
write_point_cloud("sync.ply", pcd)

# Load saved point cloud and visualize it
pcd_load = read_point_cloud("sync.ply")
draw_geometries([pcd_load])



In [8]:
# Pass xyz to Open3D.PointCloud and visualize
pcd = PointCloud()
pcd.points = Vector3dVector(xyz)
write_point_cloud("TestData/sync.ply", pcd)

True

In [16]:
# Load saved point cloud and visualize it
pcd_load = read_point_cloud(data_ply)
draw_geometries([pcd_load])

# convert Open3D.PointCloud to numpy array
xyz_load = np.asarray(pcd_load.points)
print('xyz_load')
print(xyz_load)

xyz_load
[[-3.         -3.          0.17846472]
 [-2.985      -3.          0.17440115]
 [-2.97       -3.          0.17063709]
 ...
 [ 2.97        3.          0.17063709]
 [ 2.985       3.          0.17440115]
 [ 3.          3.          0.17846472]]


In [20]:
    print("Load a ply point cloud, print it, and render it")
    pcd = read_point_cloud("TestData/sync.ply")
    print(pcd)
    print(np.asarray(pcd.points))
    draw_geometries([pcd])

    print("Downsample the point cloud with a voxel of 0.05")
    downpcd = voxel_down_sample(pcd, voxel_size = 0.05)
    draw_geometries([downpcd])

    print("Recompute the normal of the downsampled point cloud")
    estimate_normals(downpcd, search_param = KDTreeSearchParamHybrid(
            radius = 0.1, max_nn = 30))
    draw_geometries([downpcd])

    print("Print a normal vector of the 0th point")
    print(downpcd.normals[0])
    print("Print the normal vectors of the first 10 points")
    print(np.asarray(downpcd.normals)[:10,:])
    print("")


Load a ply point cloud, print it, and render it
PointCloud with 160801 points.
[[-3.         -3.          0.17846472]
 [-2.985      -3.          0.17440115]
 [-2.97       -3.          0.17063709]
 ...
 [ 2.97        3.          0.17063709]
 [ 2.985       3.          0.17440115]
 [ 3.          3.          0.17846472]]
Downsample the point cloud with a voxel of 0.05
Recompute the normal of the downsampled point cloud
Print a normal vector of the 0th point
[0.18925442 0.21238278 0.95868468]
Print the normal vectors of the first 10 points
[[ 0.18925442  0.21238278  0.95868468]
 [ 0.14832633 -0.01053055  0.9888824 ]
 [ 0.08089142 -0.06675276 -0.99448512]
 [ 0.68583492 -0.52396149 -0.50506912]
 [ 0.26488907  0.34997715  0.89852645]
 [ 0.2611241   0.21195618  0.94174773]
 [ 0.13001961  0.03876958 -0.99075316]
 [ 0.28799628  0.57241931  0.76772018]
 [ 0.08585969  0.23150057  0.96903849]
 [ 0.20777711  0.20212181  0.95706606]]



In [22]:
    print("Load a ply point cloud, print it, and render it")
    pcd = read_point_cloud("TestData/sync.ply")
    print(pcd)
    print(np.asarray(pcd.points))
    draw_geometries([pcd])

Load a ply point cloud, print it, and render it
PointCloud with 160801 points.
[[-3.         -3.          0.17846472]
 [-2.985      -3.          0.17440115]
 [-2.97       -3.          0.17063709]
 ...
 [ 2.97        3.          0.17063709]
 [ 2.985       3.          0.17440115]
 [ 3.          3.          0.17846472]]


In [23]:
    print("Downsample the point cloud with a voxel of 0.05")
    downpcd = voxel_down_sample(pcd, voxel_size = 0.05)
    draw_geometries([downpcd])

Downsample the point cloud with a voxel of 0.05


In [24]:
    print("Recompute the normal of the downsampled point cloud")
    estimate_normals(downpcd, search_param = KDTreeSearchParamHybrid(
            radius = 0.1, max_nn = 30))
    draw_geometries([downpcd])

Recompute the normal of the downsampled point cloud


In [26]:
    print("Print a normal vector of the 0th point")
    print(downpcd.normals[150])

Print a normal vector of the 0th point
[0.28118909 0.83816248 0.46735035]


In [27]:
mesh=downpcd
print("Computing normal and rendering it.")
mesh.compute_vertex_normals()
print(np.asarray(mesh.triangle_normals))
draw_geometries([mesh])

Computing normal and rendering it.


AttributeError: 'open3d.open3d.PointCloud' object has no attribute 'compute_vertex_normals'